In [13]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Load data and transform

In [14]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [15]:
temp = next(iter(trainloader))

In [16]:
abc = temp[0][0][0][:][:]


In [17]:
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


In [18]:
dataiter = iter(trainloader)
images, labels = next(dataiter)

# show images
# imshow(torchvision.utils.make_grid(images))

# Define Convolutional Model

In [19]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [20]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

# Training loop

In [21]:
num_epochs = 100
for epoch in range(num_epochs):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 300 == 0:    # print every 2000 mini-batches
            print(f'[{epoch+1}, {i:5d}] loss: {running_loss / 300:.3f}')
            running_loss = 0.0

print('Finished Training')


[1,     0] loss: 0.008
[1,   300] loss: 2.301
[1,   600] loss: 2.292
[1,   900] loss: 2.262
[1,  1200] loss: 2.175
[1,  1500] loss: 2.090
[2,     0] loss: 0.007
[2,   300] loss: 2.005
[2,   600] loss: 1.909
[2,   900] loss: 1.839
[2,  1200] loss: 1.793
[2,  1500] loss: 1.736
[3,     0] loss: 0.005
[3,   300] loss: 1.672
[3,   600] loss: 1.645
[3,   900] loss: 1.616
[3,  1200] loss: 1.580
[3,  1500] loss: 1.590
[4,     0] loss: 0.006
[4,   300] loss: 1.533
[4,   600] loss: 1.494
[4,   900] loss: 1.504
[4,  1200] loss: 1.465
[4,  1500] loss: 1.474
[5,     0] loss: 0.004
[5,   300] loss: 1.413
[5,   600] loss: 1.439
[5,   900] loss: 1.392
[5,  1200] loss: 1.389
[5,  1500] loss: 1.391
[6,     0] loss: 0.005
[6,   300] loss: 1.334
[6,   600] loss: 1.348
[6,   900] loss: 1.331
[6,  1200] loss: 1.294
[6,  1500] loss: 1.304
[7,     0] loss: 0.004
[7,   300] loss: 1.270
[7,   600] loss: 1.252
[7,   900] loss: 1.251
[7,  1200] loss: 1.246
[7,  1500] loss: 1.258
[8,     0] loss: 0.003
[8,   300] 

In [25]:
# Save model
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

RuntimeError: Parent directory models_pytorch does not exist.

# Test Loop

In [24]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    # for data in trainloader: # Test on the training set
    for data in testloader: # Test on the test set (This is what we want to do)
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct // total} %')

Accuracy of the network on the 10000 test images: 62 %
